In [1]:
import pathlib

architecture_result_paths = list(pathlib.Path("../results/gorillas").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

Number of architectures: 14


In [2]:
import json
import pandas as pd

tials_that_should_exist = []
for architecture_result_path in architecture_result_paths:
    tials_that_should_exist.append(architecture_result_path.name)
    tials_that_should_exist.append(architecture_result_path.name + "-QAT")
    tials_that_should_exist.append(architecture_result_path.name + "-QAT&KD")
    

dataframe_list = []
trials_that_exist = []
for architecture_result_path in architecture_result_paths:
    trials = list(architecture_result_path.glob("*"))
    
    for trial in trials:
        config_dict = json.loads((trial / "config.json").read_text())
        metrics_dict = json.loads((trial / "metrics.json").read_text())
        
        name = trial.parent.name
        
        if config_dict["use_quantization_aware_training"]["0"]:
            name += "-QAT"
        if config_dict["loss_mode"]["0"] == "distillation/offline/response-based":
            name += "&KD"
            
        if "aggregated/cxlkfold/val/embeddings/knn/accuracy_max" not in metrics_dict.keys():
            continue
        else:
            dataframe_entry = {
                "name": name,
                "knn-1 accuracy": metrics_dict["aggregated/cxlkfold/val/embeddings/knn/accuracy_max"]["0"],
                "knn-1 precision": metrics_dict["aggregated/cxlkfold/val/embeddings/knn/precision_max"]["0"],
                "knn-1 f1": metrics_dict["aggregated/cxlkfold/val/embeddings/knn/f1_max"]["0"],
                "knn-5 accuracy": metrics_dict["aggregated/cxlkfold/val/embeddings/knn5/accuracy_max"]["0"],
                "knn-5 precision": metrics_dict["aggregated/cxlkfold/val/embeddings/knn5/precision_max"]["0"],
                "knn-5 f1": metrics_dict["aggregated/cxlkfold/val/embeddings/knn5/f1_max"]["0"],
                "knn-5 accuracy top-5": metrics_dict["aggregated/cxlkfold/val/embeddings/knn5/accuracy_top5_max"]["0"],
                
            }
            dataframe_list.append(dataframe_entry)
            trials_that_exist.append(name)
            
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)

tails_that_are_missing = set(tials_that_should_exist) - set(trials_that_exist)
tails_that_are_missing = sorted(list(tails_that_are_missing))


missing_kd = []
missing_baseline = []
missing_qat = []
for tail in tails_that_are_missing:
    if "&KD" in tail:
        missing_kd.append(tail)
    elif "-QAT" in tail:
        missing_qat.append(tail)
    else:
        missing_baseline.append(tail)
        
with open("missing.txt", "w") as file:
    file.write("Missing KD:\n")
    for tail in missing_kd:
        file.write(tail + "\n")
        
    file.write("\nMissing QAT:\n")
    for tail in missing_qat:
        file.write(tail + "\n")
        
    file.write("\nMissing Baseline:\n")
    for tail in missing_baseline:
        file.write(tail + "\n") 
    
        

In [3]:
# get data from ptq

import pathlib
import pandas as pd
import json

dataframe_list = dataframe_list
architecture_result_paths = list(pathlib.Path("../runs").glob("*"))
print("Number of architectures:", len(list(architecture_result_paths)))

for architecture_path in architecture_result_paths:
    results = json.loads((architecture_path / "results.json").read_text())
    results = results["quantized"]
    dataframe_list.append({
        "name": architecture_path.name + "-PTQ",
        "knn-1 accuracy": results["knn1"]["accuracy"],
        "knn-1 precision": results["knn1"]["precision"],
        "knn-1 f1": results["knn1"]["f1"],
        "knn-5 accuracy": results["knn1"]["accuracy"],
        "knn-5 precision": results["knn1"]["precision"],
        "knn-5 f1": results["knn1"]["f1"],
        "knn-5 accuracy top-5": results["knn1"]["accuracy_top5"],
    })
    
dataframe = pd.DataFrame(dataframe_list)
dataframe.to_csv("gorillas.csv", index=False)

Number of architectures: 5
